In [ ]:
from google.colab import files
# uploaded = files.upload()
# file_name = "AntiScam_all.txt"
# data = uploaded[file_name].decode("latin-1").split("\r\n")

with open("AntiScam_all.txt", "rb") as f:
    contents = f.read().decode("latin-1").split("\r\n")
# print(contents)
data=contents

In [ ]:
# 0-scammer
# 1-client
# In current code, bot is the client.(1)

In [ ]:
# data[0:15]

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        # Mapping word to numeric id
        self.word2index = {}
        # Mapping numeric id to word
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [ ]:
import os
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
MAX_LENGTH = 15  # Maximum sentence length to consider
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def extractSentencePairs(data):
  qa_pairs = []
  last_bot_line=""
  last_client_line=""
  for i in range(len(data)-1):
    inputLine = data[i].split("\t")
    targetLine = data[i+1].split("\t")
    #Checking if not blank case
    if (inputLine[0]=='1' or inputLine[0]=='0') and (targetLine[0]=='1' or targetLine[0]=='0'):
      if inputLine[0]=='0' and targetLine[0]=='1':
        qa_pairs.append([inputLine[1], targetLine[1]])
        last_bot_line=inputLine[1]
      elif inputLine[0]=='1' and targetLine[0]=='0':
        qa_pairs.append([inputLine[1], targetLine[1]])
        last_client_line=inputLine[1]
      elif inputLine[0]=='1' and targetLine[0]=='1':
        qa_pairs.append([last_bot_line, targetLine[1]])
      elif inputLine[0]=='0' and targetLine[0]=='0':
        qa_pairs.append([last_client_line, targetLine[1]])
  return qa_pairs
  

# def extractSentencePairs(data):
#   qa_pairs = []
#   last_bot_line=""
#   last_client_line=""
#   for i in range(len(data)-1):
#     inputLine = data[i].split("\t")
#     targetLine = data[i+1].split("\t")
#     #Checking if not blank case
#     if (inputLine[0]=='1' or inputLine[0]=='0') and (targetLine[0]=='1' or targetLine[0]=='0'):
#       if inputLine[0]=='0' and targetLine[0]=='1':
#         qa_pairs.append([normalizeString(inputLine[1]), normalizeString(targetLine[1])])
#         last_bot_line=normalizeString(inputLine[1])
#       elif inputLine[0]=='1' and targetLine[0]=='0':
#         qa_pairs.append([normalizeString(inputLine[1]), normalizeString(targetLine[1])])
#         last_client_line=normalizeString(inputLine[1])
#       elif inputLine[0]=='1' and targetLine[0]=='1':
#         qa_pairs.append([normalizeString(last_bot_line), normalizeString(targetLine[1])])
#       elif inputLine[0]=='0' and targetLine[0]=='0':
#         qa_pairs.append([normalizeString(last_client_line), normalizeString(targetLine[1])])
#   return qa_pairs

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(corpus_name):
    # Split every line into pairs and normalize
    pairs = extractSentencePairs(data)
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name):
    voc, pairs = readVocs(corpus_name)
    pairs = filterPairs(pairs)
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    return voc, pairs

corpus_name ="testcorpus"
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus_name)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:5]:
    print(pair)


pairs:
['Hello my name is NAMESLOT and I am with Amazon customer support.', '"Yes, Hello NAMESLOT."']
['Hello my name is NAMESLOT and I am with Amazon customer support.', 'I recently purchased a heater but it has not arrived']
['I recently purchased a heater but it has not arrived', "I'm sorry to hear that."]
['I recently purchased a heater but it has not arrived', 'Can I have your name and address to look this up?']
['Can I have your name and address to look this up?', 'Why is my address necessary NAMESLOT?']


In [ ]:
# pairs

In [ ]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 2646 / 4624 = 0.5722
Trimmed from 6291 pairs to 4180, 0.6644 of total


In [ ]:
pairs[0]

['Hello my name is NAMESLOT and I am with Amazon customer support.',
 '"Yes, Hello NAMESLOT."']

In [ ]:
import re
for pair in pairs:
  pair[0]=re.sub(",", "", pair[0])
  pair[1]=re.sub(",", "", pair[1])

In [ ]:
pairs[0]

['Hello my name is NAMESLOT and I am with Amazon customer support.',
 '"Yes Hello NAMESLOT."']

In [ ]:
list_strs=[]
for pair in pairs:
  line= pair[0]+","+pair[1]
  list_strs.append(line)

In [ ]:
for l in list_strs[:10]:
  print(l)

Hello my name is NAMESLOT and I am with Amazon customer support.,"Yes Hello NAMESLOT."
Hello my name is NAMESLOT and I am with Amazon customer support.,I recently purchased a heater but it has not arrived
I recently purchased a heater but it has not arrived,I'm sorry to hear that.
I recently purchased a heater but it has not arrived,Can I have your name and address to look this up?
Can I have your name and address to look this up?,Why is my address necessary NAMESLOT?
Alright before I proceed may I have your full name for personal record.,I can give you my employee number if you need.
I can give you my employee number if you need.,"Ok what is your employee number?"
NAMESLOT,NAMESLOT.
NAMESLOT,I am seeing a customers by that name.
NAMESLOT,Did you order with a credit card?


In [ ]:
import pickle
pickle.dump(pairs, open('./pairs.pickle', 'wb'))

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 8.9 MB/s 


In [ ]:
from tokenizers import ByteLevelBPETokenizer

corpus = ""
for i in pairs:
    corpus += " ".join(i)

print(corpus[:300])
f = open('./corpus.txt', 'w')
f.write(corpus) 
f.close()

Hello my name is NAMESLOT and I am with Amazon customer support. "Yes, Hello NAMESLOT."Hello my name is NAMESLOT and I am with Amazon customer support. I recently purchased a heater but it has not arrivedI recently purchased a heater but it has not arrived I'm sorry to hear that.I recently purchased


In [ ]:
# Initialize the BPE tokenizer by huggingface
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=['./corpus.txt'], vocab_size=9000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model(".", "test_s2s")

['./test_s2s-vocab.json', './test_s2s-merges.txt']

In [ ]:
import json
crp = json.load(open('./test_s2s-vocab.json','r'))
len(crp)

3190

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# After the tokenizer is trained, we initialize the tokenizer with the created
# vocab and merges file. we specify the processing functionalities to be similar 
# to BERT. i.e we can leverage attention_masks, type_ids, ids .etc. functionalities
# with the initialized tokenizer.

# We also spicify the tokenizer to automatically pad each sentence to the max_length
# of the corpus, and enable truncation to length of 50 tokens.

# All these functionalities with just a few lines of code !!

tokenizer = ByteLevelBPETokenizer('./test_s2s-vocab.json', 
                                  './test_s2s-merges.txt')
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=1, pad_token='<pad>')

print(tokenizer.encode("hello world"))

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [ ]:
# Let's test out some functionalities of the tokenizer
print(tokenizer.encode("hello world").ids)
print(tokenizer.decode(tokenizer.encode("hello world").ids))

[0, 1074, 578, 359, 2]
<s>hello world</s>


In [ ]:

# Split pairs into 2 lists: query(X) & response(Y)
X, Y = [], []
for pair in pairs:
    X.append(pair[0])
    Y.append(pair[1])

print(len(X), len(Y))

4180 4180


In [ ]:
# We can tokenize, encode & pad all the queries & responses
# by just invoking the encode_batch function !!

X_tokenized = [i.ids for i in tokenizer.encode_batch(X)]
Y_tokenized = [i.ids for i in tokenizer.encode_batch(Y)]
print(len(X_tokenized), len(Y_tokenized))

4180 4180


In [ ]:
#Lets look at one tokenized sentence
print(X_tokenized[0])
print(tokenizer.decode(X_tokenized[0]),"\n",X[0])

[0, 580, 332, 340, 291, 456, 330, 276, 394, 362, 349, 541, 1631, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<s>Hello my name is NAMESLOT and I am with Amazon customer support.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 
 Hello my name is NAMESLOT and I am with Amazon customer support.


In [ ]:
from sklearn.model_selection import train_test_split

# Create 80:10:10 training, testing and validation splits

X_train, X_test, Y_train, Y_test = train_test_split(X_tokenized, Y_tokenized, 
                                                    test_size=0.2, random_state=42)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, 
                                                    test_size=0.5, random_state=42)

print(len(X_train), len(Y_train), len(X_valid), len(Y_valid), len(X_test), len(Y_test))

3344 3344 418 418 418 418


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Below we define a function to create train, test & valid dataloaders in Pytorch

batch_size = 128

def get_dataloader(x, y):
    x = torch.tensor(x).long()  #batch , max_len
    y = torch.tensor(y).long()  #batch , max_len
    x_lens = torch.sum(torch.tensor(x) != 1, 1) #batch
    y_lens = torch.sum(torch.tensor(y) != 1, 1) #batch

    data = TensorDataset(x, x_lens, y, y_lens)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

train_datalaoder = get_dataloader(X_train, Y_train)
valid_datalaoder = get_dataloader(X_valid, Y_valid)
test_datalaoder = get_dataloader(X_test, Y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Sanity check that the tensors returned by the dataloader are correct

for batch in train_datalaoder:
    x, x_lens, y, y_lens = batch
    print(x.shape, x_lens.shape, y.shape, y_lens.shape)
    break


torch.Size([128, 30]) torch.Size([128]) torch.Size([128, 24]) torch.Size([128])


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, device, n_layers=1, dropout=0):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.directions = 2
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
        self.device = device

    def forward(self, input_seq, input_lengths):
        # Convert word indexes to embeddings
        # input_seq: seq_len, batch
        # input_lengths: batch
        
        batch = input_seq.shape[1]
        embedded = self.embedding(input_seq)
  
        # embedded: seq_len, batch, hidden_size

        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.to("cpu"), 
                                                   batch_first=False,
                                                   enforce_sorted=False)

        # Forward pass through GRU
        hidden = torch.randn(self.n_layers*self.directions, batch, self.hidden_size).to(self.device)

        outputs, hidden = self.gru(packed, hidden)      

        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)

        # Sum bidirectional GRU outputs & hidden
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        hidden = torch.sum(hidden.view(-1, 2, batch, self.hidden_size), dim=1)

        # Return output and final hidden state
        # outputs: (seq_len, batch, hidden_size)
        # hidden: (n_layers, batch, hidden_size)

        return outputs, hidden

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.randn(hidden_size))

    def concat_score(self, hidden, encoder_output):
        # hidden: 1, batch, hidden_size
        # encoder_output: seq_len, batch, hidden_size

        repeat_decoder_hidden = hidden.repeat(encoder_output.size(0), 1, 1) #seq_len, batch, hidden_size
        # print('repeat_decoder_hidden', repeat_decoder_hidden)
        concatenate_hidden = torch.cat([repeat_decoder_hidden, encoder_output], -1) #seq_len, batch, hidden_size*2
        energy = self.attn(concatenate_hidden).tanh() #seq_len, batch, hidden_size
        # print('energy', energy)
        return torch.sum(self.v * energy, dim=-1) #seq_len, batch

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies)
        attn_energies = self.concat_score(hidden, encoder_outputs) #seq_len, batch
        # print('attn_energies', attn_energies)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.permute(1,0) #batch, seq_len

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=-1).unsqueeze(1)  #batch, 1, seq_len

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super().__init__()
        # Define layers
        self.embedding = embedding
        self.attn = Attn(hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat_layer = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # input_step: 1, batch_size
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)

        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # rnn_output: (1, batch, hidden_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)

        attn_weights = self.attn(rnn_output, encoder_outputs) # batch, 1, seq_len
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        # encoder_output: seq_len, batch, hidden_size
        context = attn_weights.bmm(encoder_outputs.permute(1, 0, 2)) # batch, 1, hidden_size

        rnn_output = rnn_output.squeeze(0) #batch, hidden_size
        context = context.squeeze(1) #batch, hidden_size
        
        concat_input = torch.cat([rnn_output, context], -1) #batch, hidden_size*2
        concat_output = torch.tanh(self.concat_layer(concat_input)) #batch, hidden_size

        # Predict next word. Note we don't apply softmax. 
        output = self.out(concat_output).unsqueeze(0)
        
        # outputs: (1, batch, num_words)
        # hidden: (n_layers, batch, hidden_size)

        # Return output and final hidden state
        return output, hidden

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.randn(hidden_size))

    def concat_score(self, hidden, encoder_output):
        # hidden: 1, batch, hidden_size
        # encoder_output: seq_len, batch, hidden_size

        repeat_decoder_hidden = hidden.repeat(encoder_output.size(0), 1, 1) #seq_len, batch, hidden_size
        # print('repeat_decoder_hidden', repeat_decoder_hidden)
        concatenate_hidden = torch.cat([repeat_decoder_hidden, encoder_output], -1) #seq_len, batch, hidden_size*2
        energy = self.attn(concatenate_hidden).tanh() #seq_len, batch, hidden_size
        # print('energy', energy)
        return torch.sum(self.v * energy, dim=-1) #seq_len, batch

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies)
        attn_energies = self.concat_score(hidden, encoder_outputs) #seq_len, batch
        # print('attn_energies', attn_energies)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.permute(1,0) #batch, seq_len

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=-1).unsqueeze(1)  #batch, 1, seq_len

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super().__init__()
        # Define layers
        self.embedding = embedding
        self.attn = Attn(hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat_layer = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # input_step: 1, batch_size
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)

        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # rnn_output: (1, batch, hidden_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)

        attn_weights = self.attn(rnn_output, encoder_outputs) # batch, 1, seq_len
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        # encoder_output: seq_len, batch, hidden_size
        context = attn_weights.bmm(encoder_outputs.permute(1, 0, 2)) # batch, 1, hidden_size

        rnn_output = rnn_output.squeeze(0) #batch, hidden_size
        context = context.squeeze(1) #batch, hidden_size
        
        concat_input = torch.cat([rnn_output, context], -1) #batch, hidden_size*2
        concat_output = torch.tanh(self.concat_layer(concat_input)) #batch, hidden_size

        # Predict next word. Note we don't apply softmax. 
        output = self.out(concat_output).unsqueeze(0)
        
        # outputs: (1, batch, num_words)
        # hidden: (n_layers, batch, hidden_size)

        # Return output and final hidden state
        return output, hidden

In [ ]:
class Seq2SeqModel(nn.Module):
    def __init__(self, embedding, hidden_size, num_words, max_target_len, device, 
                 n_layers=2, dropout=0.3, teacher_force=0.5):
        super().__init__()
        self.embedding = embedding
        self.encoder = EncoderRNN(hidden_size, self.embedding, device, 
                                  n_layers, dropout)
        self.decoder = DecoderRNN(self.embedding, hidden_size, num_words, 
                                  n_layers, dropout)
        self.teacher_force = teacher_force
        self.max_len = max_target_len
        self.device = device
    
    def forward(self, input_seq, input_lengths, target_seq):
        # input_seq: batch, seq_len 
        # target_seq: batch, seq_len 
        # input_lengths: batch

        max_target_len = target_seq.shape[1]
        batch_size = input_lengths.shape
        decoder_output_pred = []

        input_seq = input_seq.permute(1, 0) #seq_len, batch
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_lengths)

        # encoder_outputs: (seq_len, batch, hidden_size)
        # encoder_hidden: (n_layers, batch, hidden_size)

        decoder_input = target_seq.permute(1,0)[:1,:] # 1, batch
        decoder_hidden = encoder_hidden

        if random.random() < self.teacher_force:
            use_teacher_force = True
        else:
            use_teacher_force = False

        if use_teacher_force:
            # print("Using Teacher Force.\n")
            for t in range(1, max_target_len):
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                # print(encoder_outputs)

                # Teacher forcing: next input is current target
                decoder_input = target_seq.permute(1,0)[t].view(1, -1).to(self.device) # 1, batch
                decoder_output_pred.append(decoder_output)

        else:
            # print("Not Using Teacher Force.\n")
            for t in range(1, max_target_len):
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                _, topi = decoder_output.topk(1, dim=-1)
                decoder_input = topi.squeeze(-1).detach().to(self.device) # 1, batch
                decoder_output_pred.append(decoder_output)

        decoder_output_pred = torch.cat(decoder_output_pred).to(self.device)
        # seq_len, batch, words

        return decoder_output_pred

    def predict(self, input_seq, input_lengths):

        input_seq = input_seq.permute(1, 0) #seq_len, batch
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_lengths)
        decoder_input = torch.ones(1, 1, dtype=torch.long).to(self.device)
        decoder_hidden = encoder_hidden

        all_tokens = torch.zeros([0], dtype=torch.long).to(self.device)
        all_scores = torch.zeros([0]).to(self.device)

        for t in range(1, self.max_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_hidden)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=-1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # decoder_input = decoder_input.unsqueeze(0).to(self.device)
        # Return collections of word tokens and scores
        return all_tokens, all_scores


In [ ]:
# Below method helps us chat with the trained model.

def evaluateInput():
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = torch.tensor(tokenizer.encode(input_sentence).ids).unsqueeze(0)
            input_length = torch.tensor(input_sentence.shape[1]).unsqueeze(0)
            #1, seq_len

            # Evaluate sentence
            output_words,_ = model.predict(input_sentence.to(device), input_length.to(device))

            # Format and print response sentence
            output_words = output_words.squeeze().tolist()
            output_words = [x for x in output_words if x not in [0,1,2]]
            print('Bot:', tokenizer.decode(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Training function: Performs forward propagation, backpropagation & optimization.
# We also implement gradient clipping, which prevents the gradients from exploding

def train(model, dataloader, optimizer, criterion, clip, num_words, scheduler):

    model.train()

    ep_t_loss = 0
    batch_num  = 0

    for ix, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        x, x_lengths, y, y_lengths = batch
        if ix % 50 == 0:
            print("Training Batch ",ix)
        
        optimizer.zero_grad()

        output = model(x, x_lengths, y)
        # print("output ",output.shape)
        # print("y ",y.shape)
        
        loss = criterion(output.view(-1, num_words),
                               y.permute(1,0)[1:].contiguous().view(-1))
        loss.backward()

        #gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        scheduler.step()

        ep_t_loss += loss.item()
        batch_num += 1

    return ep_t_loss/batch_num

In [ ]:
# Evaluation function: Calculates loss on the validation data.

def evaluate(model, dataloader, criterion, num_words):

    model.eval()

    ep_t_loss = 0
    batch_num  = 0

    for ix, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        x, x_lengths, y, y_lengths = batch
        if ix % 50 == 0:
            print("Validation Batch ",ix)
        
        optimizer.zero_grad()

        with torch.no_grad():
            output = model(x, x_lengths, y)
            
            loss = criterion(output.view(-1, num_words),
                                  y.permute(1,0)[1:].contiguous().view(-1))

            ep_t_loss += loss.item()
            batch_num += 1

    return ep_t_loss/batch_num

In [ ]:
# Utility function to measure training time per epoch

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Initializing the model with parameters
hidden_size = 512
num_words = tokenizer.get_vocab_size()
max_target_len = len(Y_tokenized[0])
n_layers = 2
dropout = 0.2
teacher_force = 0.7
clip = 5
embedding = nn.Embedding(num_words, hidden_size).to(device)

model = Seq2SeqModel(embedding, hidden_size, num_words, max_target_len, device, 
                     n_layers, dropout, teacher_force).to(device)

In [ ]:
max_target_len

24

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,348,342 trainable parameters


In [ ]:
lr = 1e-4

# We use AdamW optimizer, an improvement over the Adam optimizer
# https://towardsdatascience.com/why-adamw-matters-736223f31b5d
# Cross Entropy loss is used as the loss function
# We also experiment with a cyclic learning rate scheduler. 
# Cyclic learning rates have demonstrated capabilities of faster convergence: https://arxiv.org/pdf/1506.01186.pdf

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss(ignore_index=1)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              base_lr=lr,
                                              max_lr=lr*5, 
                                              cycle_momentum=False)

In [ ]:
best_valid_loss = float('inf')
tot_t_loss, tot_v_loss =[],[]

In [ ]:
N_EPOCHS = 15

In [ ]:
import time
from tqdm import tqdm
for epoch in tqdm(range(N_EPOCHS)):

# We finally start training !!

    start_time = time.time()
    
    tr_l= train(model, train_datalaoder, optimizer, criterion, 
                clip, num_words, scheduler)
    tot_t_loss.append(tr_l)

    val_l = evaluate(model, valid_datalaoder, criterion, num_words)
    tot_v_loss.append(val_l)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_l < best_valid_loss:
        best_valid_loss = val_l
        torch.save(model.state_dict(), 'Chatbot_v2_least_loss.pt')
        print("\nBest Model Saved !!")
    elif epoch % 3 == 0:
        torch.save(model.state_dict(), 'Chatbot_v2_checkpoint_'+str(epoch)+'.pt')
        print("\Checkpoint Model Saved !!")
    print("\n")
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    # In information theory, perplexity is a measurement of how well a probability 
    # distribution or probability model predicts a sample. It may be used to compare 
    # probability models. A low perplexity indicates the probability distribution 
    # is good at predicting the sample.
    print(f'\tTrain Total Loss: {tr_l:.3f} | Train Perplexity: {math.exp(tr_l):.3f}')
    print(f'\tVal. Total Loss: {val_l:.3f} | Valid Perplexity: {math.exp(val_l):.3f}')
    print("_________________________________________________________________")

  0%|          | 0/15 [00:00<?, ?it/s]

Training Batch  0
Validation Batch  0


  7%|▋         | 1/15 [09:00<2:06:07, 540.54s/it]


Best Model Saved !!


Epoch: 01 | Time: 9m 0s
	Train Total Loss: 7.254 | Train Perplexity: 1413.195
	Val. Total Loss: 6.145 | Valid Perplexity: 466.530
_________________________________________________________________
Training Batch  0
Validation Batch  0


 13%|█▎        | 2/15 [17:54<1:56:14, 536.53s/it]

...

\Checkpoint Model Saved !!


Epoch: 13 | Time: 6m 13s
	Train Total Loss: 4.547 | Train Perplexity: 94.345
	Val. Total Loss: 4.598 | Valid Perplexity: 99.268
_________________________________________________________________
Training Batch  0
Validation Batch  0


 93%|█████████▎| 14/15 [1:18:29<05:33, 333.17s/it]


Best Model Saved !!


Epoch: 14 | Time: 6m 2s
	Train Total Loss: 4.343 | Train Perplexity: 76.913
	Val. Total Loss: 4.564 | Valid Perplexity: 95.923
_________________________________________________________________
Training Batch  0
Validation Batch  0


100%|██████████| 15/15 [1:25:05<00:00, 340.37s/it]



Epoch: 15 | Time: 6m 36s
	Train Total Loss: 4.415 | Train Perplexity: 82.704
	Val. Total Loss: 4.799 | Valid Perplexity: 121.418
_________________________________________________________________


In [ ]:
model.load_state_dict(torch.load('./Chatbot_v2_checkpoint_6.pt'))

FileNotFoundError: ignored

In [ ]:
evaluateInput()